In [ ]:
# =============================================================================
# SIMPSONS CHARACTER CLASSIFICATION - CNN IMPLEMENTATION
# =============================================================================

# Diese Environment-Variablen sind super wichtig für CUDA Debugging!
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Macht CUDA Fehler synchron - sehr nützlich für Debugging
os.environ["TORCH_USE_CUDA_DSA"] = "1"    # Aktiviert Device-Side Assertions für bessere Fehlermeldungen

# Standard ML/DL Libraries importieren
import torch                              # PyTorch Core Library
import torch.nn as nn                     # Neural Network Module für Layer-Definitionen
import torch.nn.functional as F           # Functional API für Activation Functions etc.
import torch.optim as optim               # Optimizer Klassen (Adam, SGD, etc.)
from torch.utils.data import DataLoader, Dataset  # Data Loading Utilities
import numpy as np                        # Numerical Operations
import cv2                               # OpenCV für Image Processing
from sklearn.model_selection import train_test_split  # Train/Test Split
from collections import Counter          # Zum Zählen von Elementen
from torchvision import transforms       # Image Transformations für Data Augmentation

# =============================================================================
# CONFIGURATION - Hier können wir später mit Hyperparameter Tuning experimentieren
# =============================================================================
IMG_SIZE = 64                           # Bildgröße - 64x64 ist gut für schnelles Training
DATA_DIR = "./archive/simpsons_dataset" # Pfad zum Dataset
MAX_IMAGES_PER_CLASS = 500              # Limitiert Bilder pro Klasse (Memory Management)
BATCH_SIZE = 8                         # Kleine Batch Size für Memory-constrained Environments
NUM_EPOCHS = 20                         # Anzahl Training Epochen
INITIAL_LR = 0.001                      # Learning Rate - könnte adaptiv gemacht werden
VALID_EXTENSIONS = {'.jpg', '.jpeg', '.png'}  # Gültige Bildformate

"""
OPTIMIERUNGSPOTENTIAL #1: CONFIGURATION
- Config in separate YAML/JSON Datei auslagern
- Hyperparameter Tuning mit Optuna implementieren
- Adaptive Learning Rate Scheduling hinzufügen
- Early Stopping basierend auf Validation Loss
"""

# =============================================================================
# DATA AUGMENTATION & PREPROCESSING PIPELINES
# =============================================================================

# Training Transformations - Data Augmentation verhindert Overfitting!
train_transforms = transforms.Compose([
    transforms.ToPILImage(),             # Numpy Array zu PIL Image (für transforms compatibility)
    transforms.Resize((IMG_SIZE, IMG_SIZE)),  # Standardisiert alle Bilder auf 64x64
    transforms.RandomHorizontalFlip(p=0.5),   # 50% Chance auf horizontalen Flip
    transforms.RandomRotation(degrees=15),     # Zufällige Rotation bis 15° (Characters bleiben erkennbar)
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Farb-Variationen
    transforms.ToTensor(),               # Konvertiert zu Tensor und normalisiert [0,1]
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalisiert zu [-1, 1]
])

# Test/Validation Transformations - Keine Augmentation für konsistente Evaluation
test_transforms = transforms.Compose([
    transforms.ToPILImage(),             # Numpy zu PIL
    transforms.Resize((IMG_SIZE, IMG_SIZE)),  # Resize auf Standard-Größe
    transforms.ToTensor(),               # Zu Tensor konvertieren
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Gleiche Normalisierung
])

"""
OPTIMIERUNGSPOTENTIAL #2: DATA AUGMENTATION
- Mixup/CutMix Augmentation hinzufügen
- AutoAugment/RandAugment implementieren
- Advanced Augmentation wie Elastic Transforms
- Test Time Augmentation (TTA) für bessere Inferenz
"""

# =============================================================================
# CUSTOM DATASET CLASS
# =============================================================================
class SimpsonsDataset(Dataset):
    """
    Custom Dataset Klasse für die Simpsons Charaktere
    Erbt von torch.utils.data.Dataset für DataLoader Kompatibilität
    """
    def __init__(self, images, labels, transform=None):
        self.images = images      # Numpy Array mit Bildern
        self.labels = labels      # Numpy Array mit Labels (Integer-kodiert)
        self.transform = transform # Optional: Transformation Pipeline

    def __len__(self):
        return len(self.images)   # Anzahl Samples im Dataset

    def __getitem__(self, idx):
        """
        Gibt ein Sample (Image, Label) für gegebenen Index zurück
        Wird vom DataLoader automatisch aufgerufen
        """
        image = self.images[idx]  # Bild an Index idx
        label = self.labels[idx]  # Entsprechendes Label
        if self.transform:        # Falls Transformationen definiert sind
            image = self.transform(image)  # Transformationen anwenden
        return image, label       # Tuple zurückgeben

"""
OPTIMIERUNGSPOTENTIAL #3: DATASET CLASS
- Memory Mapping für große Datasets
- Lazy Loading implementieren (nur bei Bedarf laden)
- Caching häufig verwendeter Samples
- Multi-threaded Data Loading optimieren
"""

# =============================================================================
# DATA LOADING & PREPROCESSING
# =============================================================================

# Alle Charakter-Ordner im Dataset Directory finden
label_names = sorted([name for name in os.listdir(DATA_DIR)
                     if os.path.isdir(os.path.join(DATA_DIR, name))])

# Anzahl Bilder pro Charakter zählen (für Top-K Auswahl)
image_counts = {label: len([f for f in os.listdir(os.path.join(DATA_DIR, label))
                           if os.path.splitext(f)[1].lower() in VALID_EXTENSIONS])
               for label in label_names}

# Top 10 Charaktere mit den meisten Bildern auswählen (Class Balance)
top_characters = [c for c, _ in Counter(image_counts).most_common(10)]

# Label-Mapping erstellen: Charakter Name -> Integer Index
label_map = {name: idx for idx, name in enumerate(top_characters)}

# Listen für Images und Labels initialisieren
images = []  # Wird alle Bilder enthalten
labels = []  # Wird entsprechende Integer-Labels enthalten

"""
OPTIMIERUNGSPOTENTIAL #4: CLASS IMBALANCE
- Weighted Loss Functions implementieren
- SMOTE oder andere Oversampling Techniken
- Focal Loss für schwierige Klassen
- Stratified Sampling in DataLoader
"""

# Bilder laden und preprocessen
for label in top_characters:                           # Für jeden der Top Charaktere
    folder_path = os.path.join(DATA_DIR, label)        # Pfad zum Charakter-Ordner
    img_count = 0                                      # Counter für Bilder pro Klasse

    for img_name in os.listdir(folder_path):           # Für jedes Bild im Ordner
        if img_count >= MAX_IMAGES_PER_CLASS:          # Max Images erreicht?
            break                                      # Nächsten Charakter

        # Prüfen ob Datei gültiges Bildformat hat
        if os.path.splitext(img_name)[1].lower() not in VALID_EXTENSIONS:
            print(f"Warning: Skipping non-image file {os.path.join(folder_path, img_name)}")
            continue                                   # Nicht-Bild Dateien überspringen

        img_path = os.path.join(folder_path, img_name) # Vollständiger Pfad zum Bild
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)   # Bild in Farbe laden

        if img is None:                                # Laden fehlgeschlagen?
            print(f"Warning: Failed to load image {img_path}")
            continue                                   # Nächstes Bild

        # OpenCV lädt in BGR, wir brauchen RGB für Konsistenz
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Grayscale Bilder zu RGB konvertieren (falls vorhanden)
        if img.ndim == 2:                              # Nur 2 Dimensionen = Grayscale
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

        # Bild auf Standard-Größe resizen (wichtig für CNN Input!)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)

        # Bildformat validieren
        if img.shape != (IMG_SIZE, IMG_SIZE, 3):       # Erwartete Shape: (64, 64, 3)
            print(f"Warning: Image {img_path} has invalid shape {img.shape}, expected ({IMG_SIZE}, {IMG_SIZE}, 3)")
            continue                                   # Ungültige Bilder überspringen

        images.append(img)                             # Bild zur Liste hinzufügen
        labels.append(label_map[label])                # Entsprechendes Integer-Label hinzufügen
        img_count += 1                                 # Counter erhöhen

# Listen zu NumPy Arrays konvertieren (effizienter für ML Operations)
try:
    images = np.array(images, dtype=np.uint8)          # uint8 für PIL Kompatibilität
    labels = np.array(labels, dtype=np.int64)          # int64 für PyTorch CrossEntropyLoss
except ValueError as e:
    # Debugging Information falls Array-Konvertierung fehlschlägt
    print(f"Error converting to NumPy array: {e}")
    print("Shapes of first few images:")
    for i, img in enumerate(images[:5]):               # Erste 5 Bilder debuggen
        print(f"Image {i}: shape {np.array(img).shape if isinstance(img, np.ndarray) else 'Not an array'}")
    print("Total images collected:", len(images))
    raise                                              # Exception weiterwerfen

# Label-Validierung (wichtig für korrekte Loss-Berechnung!)
num_classes = len(top_characters)                      # Anzahl Klassen
if labels.max() >= num_classes or labels.min() < 0:   # Labels außerhalb gültigem Bereich?
    print(f"Error: Labels contain invalid indices. Max label: {labels.max()}, Min label: {labels.min()}, Expected range: [0, {num_classes-1}]")
    raise ValueError("Invalid label indices detected")

# Arrays validieren
print(f"Images array shape: {images.shape}, Labels array shape: {labels.shape}")

"""
OPTIMIERUNGSPOTENTIAL #5: DATA LOADING
- Parallel Processing mit multiprocessing
- Progressive Image Loading (kleinere Bilder zuerst)
- Data Validation Pipeline implementieren
- Corrupted Image Detection verbessern
"""

# =============================================================================
# TRAIN/TEST SPLIT
# =============================================================================

# Stratified Split - gleiche Klassenverteilung in Train/Test
X_train, X_test, y_train, y_test = train_test_split(
    images, labels,                                    # Input Data und Labels
    test_size=0.2,                                    # 20% für Testing
    stratify=labels,                                  # Gleiche Klassenverteilung
    random_state=42                                   # Reproduzierbare Ergebnisse
)

# Dataset Objekte mit entsprechenden Transformationen erstellen
train_dataset = SimpsonsDataset(X_train, y_train, transform=train_transforms)  # Mit Augmentation
test_dataset = SimpsonsDataset(X_test, y_test, transform=test_transforms)      # Ohne Augmentation

# DataLoader für effizientes Batch Loading erstellen
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)   # Training: gemischt
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)    # Test: nicht gemischt

# Summary ausgeben
print(f"Loaded {len(images)} images across {len(top_characters)} characters")
print(f"Training set: {len(X_train)} images, Test set: {len(X_test)} images")

# =============================================================================
# CNN MODEL DEFINITION
# =============================================================================

class SimpsonsCNN(nn.Module):
    """
    Convolutional Neural Network für Simpsons Character Classification

    Architektur:
    - 3 Conv Layers mit steigender Kanalanzahl (Feature Hierarchie)
    - MaxPooling nach jedem Conv Layer (Spatial Downsampling)
    - 2 Fully Connected Layers (Classification Head)
    - Dropout für Regularisierung
    """
    def __init__(self, num_classes):
        super(SimpsonsCNN, self).__init__()

        # Convolutional Layers - extrahieren Features
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)    # 3->32 channels, 3x3 kernel
        self.pool = nn.MaxPool2d(2, 2)                            # 2x2 MaxPooling (halbiert Größe)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # 32->64 channels
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # 64->128 channels

        # Fully Connected Layers - für finale Klassifikation
        self.fc1 = nn.Linear(128 * 8 * 8, 512)                   # 128*8*8 nach 3x MaxPool von 64x64
        self.dropout = nn.Dropout(0.5)                           # 50% Dropout gegen Overfitting
        self.fc2 = nn.Linear(512, num_classes)                   # Output Layer

    def forward(self, x):
        """
        Forward Pass durch das Netzwerk
        Input: x mit Shape (batch_size, 3, 64, 64)
        Output: Logits mit Shape (batch_size, num_classes)
        """
        # Conv Block 1: Conv -> ReLU -> MaxPool (64x64 -> 32x32)
        x = self.pool(F.relu(self.conv1(x)))

        # Conv Block 2: Conv -> ReLU -> MaxPool (32x32 -> 16x16)
        x = self.pool(F.relu(self.conv2(x)))

        # Conv Block 3: Conv -> ReLU -> MaxPool (16x16 -> 8x8)
        x = self.pool(F.relu(self.conv3(x)))

        # Flatten für FC Layers: (batch, 128, 8, 8) -> (batch, 128*8*8)
        x = x.view(-1, 128 * 8 * 8)

        # FC Block 1: Linear -> ReLU -> Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        # Output Layer: Linear (keine Activation, da CrossEntropyLoss LogSoftmax eingebaut hat)
        x = self.fc2(x)
        return x

"""
OPTIMIERUNGSPOTENTIAL #6: MODEL ARCHITECTURE
- ResNet/DenseNet Connections für tiefere Netzwerke
- Attention Mechanisms hinzufügen
- EfficientNet als Backbone verwenden
- Transfer Learning mit vortrainierten Models
- Batch Normalization zwischen Layers
"""

# =============================================================================
# DEVICE SETUP & MODEL INITIALIZATION
# =============================================================================

# Debugging Information für verfügbare Hardware
print(f"PyTorch version: {torch.__version__}")              # PyTorch Version
print(f"CUDA available: {torch.cuda.is_available()}")       # CUDA verfügbar?
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")            # CUDA Version
    try:
        print(f"GPU device: {torch.cuda.get_device_name(0)}")  # GPU Name
    except Exception as e:
        print(f"Error getting GPU device name: {e}")
print(f"MPS available: {torch.backends.mps.is_available()}")  # Apple Metal Performance Shaders

# Device Selection mit robuster Fallback-Logik
device = torch.device("cpu")                                # Standard: CPU
if torch.cuda.is_available():                               # CUDA verfügbar?
    try:
        torch.cuda.init()                                   # CUDA initialisieren
        test_tensor = torch.ones(1, device="cuda")          # Test-Tensor auf GPU
        # device = torch.device("cuda")
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # GPU verwenden
        print("CUDA initialized successfully, using GPU.")
    except Exception as e:
        print(f"CUDA initialization failed: {e}. Falling back to CPU.")
else:
    print("No GPU available or CUDA initialization failed, using CPU.")

# Model initialisieren und auf gewähltes Device verschieben
try:
    model = SimpsonsCNN(num_classes=len(top_characters)).to(device)  # Model auf Device
    test_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE, device=device)  # Test Input
    model(test_input)                                       # Forward Pass testen
    print(f"Model successfully moved to {device} and tested.")
except Exception as e:
    # Fallback zu CPU falls GPU-Initialisierung fehlschlägt
    print(f"Error moving model to device {device}: {e}")
    print("Falling back to CPU.")
    device = torch.device("cpu")
    model = SimpsonsCNN(num_classes=len(top_characters)).to(device)
    test_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE, device=device)
    model(test_input)
    print("Model successfully moved to CPU and tested.")

# Loss Function und Optimizer initialisieren
criterion = nn.CrossEntropyLoss()                          # Standard für Multi-Class Classification
optimizer = optim.Adam(model.parameters(), lr=INITIAL_LR)   # Adam Optimizer
print(f"Using {device} device for training")

"""
OPTIMIERUNGSPOTENTIAL #7: TRAINING SETUP
- Learning Rate Scheduler implementieren
- Different Optimizers ausprobieren (AdamW, SGD mit Momentum)
- Mixed Precision Training für GPU Speedup
- Gradient Accumulation für größere effektive Batch Size
- Model Checkpointing implementieren
"""

# =============================================================================
# TRAINING LOOP
# =============================================================================

for epoch in range(NUM_EPOCHS):                            # Für jede Epoche
    model.train()                                          # Training Mode (Dropout aktiv)
    running_loss = 0.0                                     # Loss Accumulator

    for batch_idx, (inputs, labels) in enumerate(train_loader):  # Für jeden Batch
        try:
            # Tensors auf korrektes Device verschieben
            inputs = inputs.to(device, non_blocking=True)   # Non-blocking für Speedup
            labels = labels.to(device, non_blocking=True)

            # Device und Label Validierung (wichtig für Debugging!)
            if inputs.device != device or labels.device != device:
                raise RuntimeError(f"Tensor device mismatch: inputs on {inputs.device}, labels on {labels.device}, expected {device}")
            if labels.max() >= num_classes or labels.min() < 0:
                raise RuntimeError(f"Invalid label values in batch: max {labels.max()}, min {labels.min()}, expected [0, {num_classes-1}]")

            # Standard Training Steps
            optimizer.zero_grad()                          # Gradienten zurücksetzen
            outputs = model(inputs)                        # Forward Pass
            loss = criterion(outputs, labels)              # Loss berechnen

            # Loss Validierung
            if loss.device != device:
                raise RuntimeError(f"Loss on incorrect device: {loss.device}, expected {device}")
            if torch.isnan(loss) or torch.isinf(loss):
                raise RuntimeError(f"Invalid loss value: {loss.item()}")

            # Gradient Clipping gegen exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            loss.backward()                                # Backward Pass (Gradienten berechnen)
            optimizer.step()                               # Parameter updaten
            running_loss += loss.item()                    # Loss akkumulieren

        except Exception as e:
            # Robuste Fehlerbehandlung mit CPU Fallback
            print(f"Error in batch {batch_idx+1}, epoch {epoch+1}: {e}")
            print(f"Inputs device: {inputs.device}, Labels device: {labels.device}, Model device: {next(model.parameters()).device}")
            print(f"Input shape: {inputs.shape}, Label shape: {labels.shape}, Label values: {labels.tolist()}")
            print("Falling back to CPU for this batch.")

            # Temporär zu CPU wechseln
            model.to("cpu")
            inputs = inputs.to("cpu")
            labels = labels.to("cpu")
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            model.to(device)  # Zurück zum ursprünglichen Device

    # Epoch Summary
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {running_loss/len(train_loader):.4f}, Learning Rate: {INITIAL_LR:.6f}")

    # =============================================================================
    # EVALUATION ON TEST SET
    # =============================================================================

    model.eval()                                          # Evaluation Mode (Dropout deaktiviert)
    correct = 0                                           # Korrekte Vorhersagen zählen
    total = 0                                             # Gesamtanzahl Samples

    with torch.no_grad():                                 # Keine Gradienten für Evaluation
        for inputs, labels in test_loader:                # Für jeden Test Batch
            inputs = inputs.to(device, non_blocking=True)  # Auf Device verschieben
            labels = labels.to(device, non_blocking=True)
            outputs = model(inputs)                       # Forward Pass
            _, predicted = torch.max(outputs.data, 1)     # Klasse mit höchster Wahrscheinlichkeit
            total += labels.size(0)                       # Batch Size addieren
            correct += (predicted == labels).sum().item() # Korrekte Vorhersagen zählen

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

"""
OPTIMIERUNGSPOTENTIAL #8: TRAINING & EVALUATION
- Validation Set für besseres Monitoring
- Early Stopping implementieren
- Learning Rate Scheduling
- Confusion Matrix und Per-Class Metrics
- Tensorboard/Wandb Logging
- Model Ensemble für bessere Performance
"""

# =============================================================================
# MODEL SAVING
# =============================================================================

#torch.save(model.state_dict(), "simpsons_cnn.pth")        # Nur Model Weights speichern
#print("Model saved as simpsons_cnn.pth")

"""
FINALES OPTIMIERUNGSPOTENTIAL:
1. Config Management: YAML/JSON configs
2. Logging: Structured logging mit wandb/tensorboard
3. Model Versioning: MLflow für Experiment Tracking
4. Data Pipeline: mehr robuste Datenvalidierung
5. Architecture: moderne Architekturen wie EfficientNet
6. Training: Advanced Training Techniques (Mixed Precision, etc.)
7. Deployment: Model Serving Pipeline
8. Monitoring: Performance Monitoring in Production
"""